In [1]:
from donation_detector_parallel import * 

In [3]:
find_donation_option('childrenshealthdefense.org')

childrenshealthdefense.org
Donation option found: donate; https://childrenshealthdefense.org/about-us/donate/


(1, 'donate', 'https://childrenshealthdefense.org/about-us/donate/')

In [ ]:
start = 0
end = 100
df_domain_partition = df_domain[start:end]
df_domain_partition['donation'] = df_domain_partition['domain'].apply(find_donation_option)
df_domain_partition.to_csv(f'donation_df/{start}_{end}.csv')


In [5]:
from bs4 import BeautifulSoup
import requests

In [29]:
keywords = ["donation", "donate", 'support us', 'support','buy me a coffee', 'tip']

def detect_keywords(soup):
    elements_with_keywords = []
    for keyword in keywords:
        elements_with_keywords.extend(soup.find_all(text=lambda text: text and keyword in text.lower()))
        elements_with_keywords.extend(soup.find_all("a", href=lambda href: href and any(keyword in href.lower() for keyword in keywords)))
        elements_with_keywords.extend(soup.find_all("button", text=lambda text: text and any(keyword in text.lower() for keyword in keywords)))
    for element in elements_with_keywords:
        if clean_string(element.text) == '' or len(clean_string(element.text).split(' ')) > 7:
            elements_with_keywords.remove(element)
    return elements_with_keywords

def find_donation_bs(url):
    if not url.startswith('https://'):
        url = 'https://' + url

    response = requests.get(url)
    html_content = response.text
    soup = BeautifulSoup(html_content, "html.parser")
    
    elements_with_keywords = detect_keywords(soup)

    iframes = soup.find_all("iframe")

    for iframe in iframes:
        iframe_src = iframe.get("src")
        if iframe_src:
            # Make a separate HTTP request to the URL specified in the src attribute
            iframe_response = requests.get(iframe_src)
            iframe_content = iframe_response.text

            # Parse the content of the iframe using Beautiful Soup
            iframe_soup = BeautifulSoup(iframe_content, "html.parser")

            # Perform your detection logic here (e.g., find elements with keywords)
            elements_with_keywords_iframe = detect_keywords(iframe_soup)
            elements_with_keywords.extend(elements_with_keywords_iframe)
        else:
            print("No src attribute found for iframe")
    return elements_with_keywords




In [36]:
start = 0
end = 10
df_domain = pd.read_csv('ideo_domain_mbfc081123_with_weights.tsv', sep='\t')
df_domain = df_domain[start: end]